# 🧪 Lab 3: Use an MCP Server as a Plugin in a Semantic Kernel Agent

In this lab, you'll learn how to **extend a Semantic Kernel agent** by connecting it to a **MCP server**. MCP (Model Context Protocol) allows agents to invoke external tools, services, or other agents as plugins.

You'll specifically:
- Connect to **your own Codebeamer MCP server** (which build in **Lab2**) as a tool via `MCPStreamableHttpPlugin`
- Use this plugin inside a **Semantic Kernel agent**
- Interact with the agent through a chat interface
- See the agent automatically call functions on the Codebeamer MCP server to answer questions

This lab showcases how Semantic Kernel can leverage **modular, tool-augmented AI workflows** by treating external MCP servers as powerful extensions to the agent's reasoning capabilities.


In [1]:
import os

from dotenv import load_dotenv

from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin
from semantic_kernel.agents import AzureResponsesAgent

# Setup
load_dotenv(override=True)

True

In [2]:
# Check environment variables
print("AZURE_OPENAI_ENDPOINT:", os.getenv("AZURE_OPENAI_ENDPOINT"))
print("AZURE_OPENAI_KEY:", "***" if os.getenv("AZURE_OPENAI_KEY") else "Not set")
print("MODEL_DEPLOYMENT_NAME:", os.getenv("MODEL_DEPLOYMENT_NAME"))
print("MODEL_DEPLOYMENT_API_VERSION:", os.getenv("MODEL_DEPLOYMENT_API_VERSION"))

AZURE_OPENAI_ENDPOINT: https://sunhu-mhjykg2a-swedencentral.cognitiveservices.azure.com/openai/responses?api-version=2025-04-01-preview
AZURE_OPENAI_KEY: ***
MODEL_DEPLOYMENT_NAME: gpt-5-mini
MODEL_DEPLOYMENT_API_VERSION: 2025-04-01-preview


The code block below initializes and connects to a Codebeamer MCP (Model Context Protocol) plugin using the `MCPStreamableHttpPlugin` interface.

- `MCPStreamableHttpPlugin` this enable connections to MCP Servers through HTTPS.
- start the MCP server built in **Lab2** first.

In [3]:
codebeamer_plugin = MCPStreamableHttpPlugin(
    name="Codebeamer",
    description="Codebeamer Plugin",
    url="http://localhost:8080/mcp",
)

# Start the connection to the MCP plugin
await codebeamer_plugin.connect()

- Description of the following code

In [4]:
# Create the client using API key and endpoint
client = AzureResponsesAgent.create_client(
    deployment_name=os.getenv("MODEL_DEPLOYMENT_NAME"),
    api_version=os.getenv("MODEL_DEPLOYMENT_API_VERSION"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_KEY"),
)

# Create the agent with the chat service and plugin
codebeamer_agent = AzureResponsesAgent(
    ai_model_id=os.getenv("MODEL_DEPLOYMENT_NAME"),
    client=client,
    name="CodebeamerAgent",
    description="A chat bot that helps users interact with Codebeamer.",
    instructions="""
You are a chat bot. And you help users interact with Codebeamer.
You are especially good at answering questions about the Microsoft semantic-kernel project.
You can call functions to get the information you need.
""",
    plugins=[codebeamer_plugin],
)

- Description of the following code


In [5]:
# Create the thread - starts as None, will be set after first response
thread = None

# Run the agent as a chat
while True:
    user_input = input("You: ")
    
    if user_input.lower() in ["exit", "quit"]:
        break

    # Get response from the agent
    response = await codebeamer_agent.get_response(
        messages=user_input,
        thread=thread,
    )

    # Print the response
    print(f"Bot: {response.content}\n")
    
    # Update the thread with the response to maintain conversation context
    thread = response.thread

await codebeamer_plugin.close()

Bot: I found these projects in Codebeamer:

- ID: 19282 — Name: Global Hackathon

What would you like to do next?
- List trackers for project 19282 (I can fetch them now)
- Get tracker items for a specific tracker
- Post a comment on a tracker item
- Anything else (tell me what you need)

Reply with which action (for example: "List trackers for 19282").

Bot: I found 17 trackers in project 19282:

- 34688470 — Contacts
- 34688471 — Releases
- 34688472 — User Stories
- 34688473 — Risks
- 34688474 — Teams
- 34688475 — Customer Requirement Specifications
- 34688476 — System Requirement Specifications
- 34688477 — Change Requests
- 34688478 — Bugs
- 34688479 — Tasks
- 34688480 — Test Cases
- 34688481 — Test Sets
- 34688482 — Test Configurations
- 34688483 — Test Runs
- 34688484 — Timekeeping
- 34772813 — 法規要件
- 34772901 — 仕様

What would you like to do next? For example:
- "List items for tracker 34688478" (Bugs)
- "Get items for 34688472" (User Stories)
- "Get tracker details for 34688480"

### 🧪 Try it out!

In this lab, you used the GitHub MCP Server — but the real power comes from the flexibility of the `MCPStdioPlugin` in Semantic Kernel. This component allows you to connect to **any tool that implements the Model Context Protocol (MCP)**.

You can easily swap in other MCP-compatible servers, or even chain multiple plugins together to create powerful tool-augmented agents.

---

🧰 **Explore These MCP Plugin Servers:**

- 🔗 [Official MCP Server Integrations (GitHub)](https://github.com/modelcontextprotocol/servers?tab=readme-ov-file#%EF%B8%8F-official-integrations)  
  A growing list of plugins including GitHub, Weather, Jira, Open Interpreter, and more.

- 📚 [10 Must-Know MCP Servers for Developers (DevShorts)](https://www.devshorts.in/p/ten-must-know-mcp-servers-for-every?utm_source=chatgpt.com)  
  A curated blog post with descriptions, commands, and usage tips.
